In [28]:
#for saving or loading the model as required using pkl
# import pickle
# pickle_out = open("model.pkl","wb")
# pickle.dump(model,pickle_out)
# pickle_out.close()
# from google.colab import drive
# drive.mount('/content/drive')
# with open('/content/model.pkl', 'w') as f:
#   f.write('/content/drive/MyDrive/model_2k')

# ###### Loading or Saving model

import torch
#for saving or loading the model as required using PyTorch
# torch.save(model, '/content/drive/MyDrive/model_2000_manually_ROW_WISE')
model = torch.load('/content/drive/MyDrive/model_2000_manually_ROW_WISE')


In [29]:
#code to clean new summaries text
import nltk
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

nltk.download("punkt")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
sentence = "i am a first year full-time mba student at the university of california irvine, paul merage school of business and my focus is entrepreneurship. i worked for east japan railway company for 4 years which is the largest railway company in the world. with knowledge of finance, general management skills, negotiation strategies and my engineering background, i would like to contribute to provide products and services to national and international market players.my specialties include: anaylytical thinking, microsoft power point, microsoft excel, microsoft word, i work for accenture as management consulting manager, focusing on oil and chemical industry. i was involved in making iot strategy for plant maintenance including erp system integration and rpa implementation. with knowledge of finance, general management skills, negotiation strategies and my engineering background, i would like to contribute to provide products and services to national and international market players. my specialties include: anaylytical thinking, sap pm, meridium, rpa, microsoft power point, microsoft excel, microsoft word, i work for accenture as management consultant, focusing on oil and chemical industry. i was involved in making iot strategy for plant maintenance including erp system integration. with knowledge of finance, general management skills, negotiation strategies and my engineering background, i would like to contribute to provide products and services to national and international market players. my specialties include: anaylytical thinking, sap pm, meridium, microsoft power point, microsoft excel, microsoft word, mba candidate class of 2016, the paul merage school of business, uc irvine, accenture - management consulting manager"

In [31]:
sentence=clean_text(sentence)

In [32]:
sentence

'i am a first year fulltime mba student at the university of california irvine  paul merage school of business and my focus is entrepreneurship i worked for east japan railway company for 4 years which is the largest railway company in the world with knowledge of finance  general management skills  negotiation strategies and my engineering background  i would like to contribute to provide products and services to national and international market playersmy specialties include anaylytical thinking  microsoft power point  microsoft excel  microsoft word  i work for accenture as management consulting manager  focusing on oil and chemical industry i was involved in making iot strategy for plant maintenance including erp system integration and rpa implementation with knowledge of finance  general management skills  negotiation strategies and my engineering background  i would like to contribute to provide products and services to national and international market players my specialties incl

In [33]:
sentence=' '.join(sentence.split())
sentence

'i am a first year fulltime mba student at the university of california irvine paul merage school of business and my focus is entrepreneurship i worked for east japan railway company for 4 years which is the largest railway company in the world with knowledge of finance general management skills negotiation strategies and my engineering background i would like to contribute to provide products and services to national and international market playersmy specialties include anaylytical thinking microsoft power point microsoft excel microsoft word i work for accenture as management consulting manager focusing on oil and chemical industry i was involved in making iot strategy for plant maintenance including erp system integration and rpa implementation with knowledge of finance general management skills negotiation strategies and my engineering background i would like to contribute to provide products and services to national and international market players my specialties include anaylyti

In [34]:
def get_sen(sentence):
  pred_data,model_output=model.predict([sentence])
  df=pd.DataFrame(pred_data)
  df=df.transpose()
  df.columns=['AA']
  df['AA']=df['AA'].astype(str)
  df['AA']=df['AA'].str[2:-2]
  df[['Word', 'Pred']] = df['AA'].str.split("': '", 1, expand=True)
  df=df.drop(columns='AA')
  return df

In [38]:
def give_dataframe_return_list_of_TAGS(temp_data):
  ccc=0
  tagg_per_sent = []
  prev_tag="O"
  string=""
  for i in range(0,len(temp_data)):
    value=temp_data['Pred'][i]
    key=temp_data['Word'][i]
    if(value=="B-ORG"):
      # print(key)
      if (prev_tag=="B-ORG" or prev_tag=="I-ORG" or prev_tag=="O"):
            if(len(string)>0):
              tagg_per_sent.append(string)
            string=key
      prev_tag="B-ORG"
    
    elif (value=="I-ORG"):
      string+=" "+key
      prev_tag="I-ORG"
      ccc+=1
    else:
      if(len(string)>0):
          tagg_per_sent.append(string)
      string=""
      prev_tag="O"
    if(len(string)>0):
        tagg_per_sent.append(string)
  # if(ccc>=1):
  #   print("i-org-count",ccc)
  return tagg_per_sent

In [40]:
length_shouldbe = len(sentence) #no of characters
temp_data=get_sen(sentence) 
len_till_now = temp_data['Word'].str.len().sum()+1+len(temp_data)
print(len_till_now)

while (len_till_now<length_shouldbe):
  aaa = get_sen(sentence[len_till_now:])
  kkk = temp_data.append(aaa)
  temp_data = kkk.reset_index(drop=True)
  len_till_now = temp_data['Word'].str.len().sum()+1+len(temp_data)
  
print(list(set(give_dataframe_return_list_of_TAGS(temp_data))))
  

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

764


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

i-org-count 9
['microsoft word', 'accenture', 'microsoft excel', 'microsoft', 'microsoft power']
